In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error

### Задание 1

#### Создание тестового набора данных

Данный датасет представляет собой информацию о параметрах жилого помещения и его стоимости. Причём так сложилось, что на стоимость данных жилых помещений оказывают большое влияние 2 параметра: количество комнат и возраст помещения. Оставшиеся два параметра (площадь и количество гаражей) несущественны.

In [2]:
samples = 100

square = np.random.randint(35, 120, samples)
num_rooms = np.random.randint(1, 5, samples)
num_garage = np.random.randint(0, 2, samples)
age = np.random.randint(1, 100, samples)

price =  num_rooms * age + num_rooms ** 2

data = pd.DataFrame({"square": square, "num_rooms": num_rooms, "num_garage": num_garage, "age": age, "price": price})

In [3]:
data.describe()

,square,num_rooms,num_garage,age,price
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,76.890000,2.590000,0.530000,49.970000,135.620000
std,23.897464,1.035677,0.501614,28.657312,93.248422
min,35.000000,1.000000,0.000000,1.000000,2.000000
25%,56.000000,2.000000,0.000000,26.250000,65.000000
50%,77.000000,3.000000,1.000000,47.500000,111.500000
75%,94.500000,3.000000,1.000000,76.000000,202.500000
max,116.000000,4.000000,1.000000,99.000000,384.000000


#### Поиск модели

In [4]:
model = LinearRegression()

Т.к. мы "не знаем" закона, которому подчиняются данные для предсказания цены, то сделаем первое предположение, что цена в какой-то мере зависит от площади и количества комнат.

In [5]:
X = data[["square", "num_rooms"]]
Y = data["price"]

model.fit(X, Y)
predictions = model.predict(X)

print("Точность предсказания: ", round(model.score(X, Y), 4))
print("MAE: ", round(mean_absolute_error(predictions, Y), 4))
print("Веса модели: ", [round(weight, 4) for weight in model.coef_])

Точность предсказания:  0.3306
MAE:  63.3077
Веса модели:  [-0.3594, 50.5916]


Как видно из результатов, наша модель имеет малую точность предсказания (всего 33%). Также параметр МАЕ является не очень удовлетворительным, т.к. при mean/min/max равным 136/2/384 в среднем ошибаться на 63 довольно плохо.

Попробуем, например, заменить параметр количество комнат на возраст помещения.

In [6]:
X = data[["square", "age"]]
Y = data["price"]

model.fit(X, Y)
predictions = model.predict(X)

print("Точность предсказания: ", round(model.score(X, Y), 4))
print("MAE: ", round(mean_absolute_error(predictions, Y), 4))
print("Веса модели: ", [round(weight, 4) for weight in model.coef_])

Точность предсказания:  0.539
MAE:  49.1333
Веса модели:  [-0.1904, 2.3667]


Результаты немного улучшились, но, по-прежнему, являются неудовлетворительными.

Возьмём предельный случай и предположим, что цена зависит от всех признаков.

In [7]:
X = data[["square", "num_rooms", "num_garage", "age"]]
Y = data["price"]

model.fit(X, Y)
predictions = model.predict(X)

print("Точность предсказания: ", round(model.score(X, Y), 4))
print("MAE: ", round(mean_absolute_error(predictions, Y), 4))
print("Веса модели: ", [round(weight, 4) for weight in model.coef_])

Точность предсказания:  0.9064
MAE:  22.3035
Веса модели:  [-0.0249, 54.6163, -4.0889, 2.483]


В этот раз результаты намного лучше. Их в принципе можно считать удовлетворительными, даже без учёта ввода дополнительных параметров. Но можно ли улучшить модель? Попробуем для начала воспользоваться полиномиальными функциями.

In [8]:
pf = PolynomialFeatures(2)

X = pf.fit_transform(data[["square", "num_rooms", "num_garage", "age"]])
Y = data["price"]

model.fit(X, Y)

print("Варианты коэффициентов: ", pf.get_feature_names())
print("Значения коэффициентов: ", [round(weight) for weight in model.coef_])

Варианты коэффициентов:  ['1', 'x0', 'x1', 'x2', 'x3', 'x0^2', 'x0 x1', 'x0 x2', 'x0 x3', 'x1^2', 'x1 x2', 'x1 x3', 'x2^2', 'x2 x3', 'x3^2']
Значения коэффициентов:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]


Как видно из расчётов, следующие комбинации коэффициентов: 1 * x1 * x2 и 1 * x2 ** 2 (т.е. 1 * num_rooms * age и num_rooms ** 2) - являются существенными для данного датасета. Попробуем добавить в наш датасет два новых параметра: num_rooms_age и num_rooms_2.

In [9]:
data["num_rooms_age"] = data["num_rooms"] * data["age"]
data["num_rooms_2"] = data["num_rooms"] ** 2

Теперь сделаем предположение, что цена зависит только от двух новых параметров.

In [10]:
X = data[["num_rooms_age", "num_rooms_2"]]
Y = data["price"]

model.fit(X, Y)
predictions = model.predict(X)

print("Точность предсказания: ", round(model.score(X, Y), 4))
print("MAE: ", round(mean_absolute_error(predictions, Y), 4))
print("Веса модели: ", [round(weight, 4) for weight in model.coef_])

Точность предсказания:  1.0
MAE:  0.0
Веса модели:  [1.0, 1.0]


Как видно из последних расчётов, предсказания полностью совпадают с реальными ценами. Таким образом, если бы мы могли каким-либо образом правильно предположить новые параметры, то получили бы аналогичные результаты без полиномиальных функций.